# 🧪 LLM with Gradio

## 🧠 Introduction

Large Language Models (LLMs) have transformed the way we build intelligent applications by enabling natural language understanding and generation. However, to make these models accessible to non-technical users, intuitive interfaces are crucial.

This notebook demonstrates how to integrate an LLM with **Gradio**, a Python library that allows developers to quickly create interactive web interfaces for machine learning models. By wrapping the model with a user-friendly UI, the system enables real-time interaction and testing without writing front-end code.

Key features of this implementation include:
- A simple chat interface for interacting with the LLM.
- Easy-to-use layout powered by Gradio components.
- Real-time response generation for input prompts.

This setup serves as a lightweight prototype or demo tool for showcasing LLM capabilities in an interactive way.


In [1]:
import os
from anthropic import Anthropic
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from typing import List

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:8]}")
else:
    print("Google API Key not set")

Anthropic API Key exists and begins sk-ant-
Deepseek API Key exists and begins sk-fe5ae


In [3]:
deepseek_client = OpenAI(base_url = "https://api.deepseek.com",api_key = deepseek_api_key)
anthropic_client = Anthropic()
llama_client = OpenAI(base_url = 'http://localhost:11434/v1', api_key='ollama')

In [4]:
system_message = "You are a helpful assistant"

In [12]:
def message_llama(prompt):
    messages = [
        {'role':'system','content':system_message},
        {'role':'user','content':prompt}
    ]
    response = llama_client.chat.completions.create(
        model = 'llama3.2',
        messages = messages,
        temperature = 0
    )

    return response.choices[0].message.content

In [13]:
message_llama("What is today's date?")

'I\'m not aware of the current date, as I\'m a large language model, I don\'t have real-time access to the current date and time. However, I can suggest ways for you to find out the current date.\n\nYou can check your device\'s clock or calendar app, or search online for "current date" to get the latest information.'

In [20]:
def shout(text):
    print(f"Shout has been called with input '{text}'")
    return text.upper()

In [21]:
shout('hi')

Shout has been called with input 'hi'


'HI'

In [28]:
gr.Interface(fn = shout, inputs = 'textbox', outputs = 'textbox').launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Shout has been called with input 'Hi'


In [32]:
gr.Interface(fn = shout, inputs = 'textbox', outputs = 'textbox',flagging_mode = 'never').launch(share = True) #Creates a public url

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://028e776128492b34c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Shout has been called with input 'Hi'


In [34]:
gr.Interface(fn = shout, inputs = 'textbox', outputs = 'textbox',flagging_mode = 'never').launch(inbrowser = True) #Automatically opens new tab

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Shout has been called with input 'Hi'
Shout has been called with input 'Hi'


In [35]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Shout has been called with input 'Hi'


In [36]:
view = gr.Interface(fn = shout,
                    inputs = gr.Textbox(label = 'Your Message',lines = 6),
                    outputs = gr.Textbox(label = 'Response',lines = 8),
                    flagging_mode = 'never'
                   )
view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Shout has been called with input 'Hi'
Shout has been called with input 'hi'


In [37]:
view = gr.Interface(fn = message_llama,
                    inputs = gr.Textbox(label = 'Your Message',lines = 6),
                    outputs = gr.Textbox(label = 'Response',lines = 8),
                    flagging_mode = 'never'
                   )
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [40]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_llama,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share = True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://363702c40b106cd6d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [46]:
def stream_llama(prompt):
    messages = [
        {'role':'system','content':system_message},
        {'role':'user','content':prompt}
    ]
    response = llama_client.chat.completions.create(
        model = 'llama3.2',
        messages = messages,
        temperature = 0,
        stream = True
    )
    reply = ""
    for chunk in response:
        reply += chunk.choices[0].delta.content or ''
        yield reply

In [47]:
view = gr.Interface(
    fn=stream_llama,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [57]:
def stream_claude(prompt):
    messages = [
        {'role':'user','content':prompt}
    ]
    response = anthropic_client.messages.stream(
        model = 'claude-3-haiku-20240307',
        max_tokens = 4096,
        messages = messages,
        temperature = 0,
        system = system_message
    )
    reply = ""
    with response as stream:
        for text in stream.text_stream:
            reply += text or ''
            yield reply

In [58]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


In [59]:
def stream_deepseek(prompt):
    messages = [
        {'role':'system','content':system_message},
        {'role':'user','content':prompt}
    ]
    response = deepseek_client.chat.completions.create(
        model = 'deepseek-chat',
        messages = messages,
        temperature = 0,
        stream = True
    )
    reply = ""
    for chunk in response:
        reply += chunk.choices[0].delta.content or ''
        yield reply

In [60]:
view = gr.Interface(
    fn=stream_deepseek,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [62]:
def stream_model(prompt, model):
    if model=="Deepseek":
        result = stream_deepseek(prompt)
    elif model=="Llama":
        result = stream_llama(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [65]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"),gr.Dropdown(['Llama','Deepseek','Claude'],label = 'Select Model',value = 'Llama')],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
